In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import firedrake
from firedrake import inner, grad, dx, assemble, Constant

### Nonlinear problems

The syntax for solving nonlinear problems is very similar to that of linear problems.
For comparison, we'll again solve the linear Poisson equation with right-hand side $x^2 - y^2$.
Instead of the `a == L` syntax we used before where `a` represented the stiffness matrix and `L` the right-hand side vector, we'll instead just write it as a big nonlinear equation `F == 0`.

In [ ]:
mesh = firedrake.UnitSquareMesh(20, 20)
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)
v = firedrake.TestFunction(Q)
bc = firedrake.DirichletBC(Q, 0., 'on_boundary')

In [ ]:
x, y = firedrake.SpatialCoordinate(mesh)
f = x**2 - y**2

In [ ]:
u_l = firedrake.Function(Q)
k = firedrake.Constant(1.)
F_l = (k * inner(grad(u_l), grad(v)) - f * v) * dx

In [ ]:
firedrake.solve(F_l == 0, u_l, bc)

Now let's solve a nonlinear problem.
In order to make the input data interesting enough, we'll first calculate the mean-square average of the gradient of the linear solution, which we're storing in the variable `g`.

In [ ]:
import numpy as np
area = assemble(Constant(1) * dx(mesh))
g = np.sqrt(assemble(inner(grad(u_l), grad(u_l)) * dx) / area)

Now we'll solve a problem where the diffusivity is nonlinear:

$$\int_\Omega\left(k\nabla u \cdot \nabla v - f\cdot v\right)dx = 0$$

for all $v$, where now

$$k = 1 + g^{-2}|\nabla u|^2.$$

Since the diffusivity is overall higher for the nonlinear problem, we should see a less pronounced solution.

In [ ]:
u = firedrake.Function(Q)
L = 1 / g
k = 1 + Constant(L)**2 * inner(grad(u), grad(u))
F = (k * inner(grad(u), grad(v)) - f * v) * dx

Under the hood, Firedrake is calling out to nonlinear solvers in the library [PETSc](https://www.mcs.anl.gov/petsc/), specifically a Newton line-search procedure.
The rough idea is that, given a guess $u_n$ for the solution, we compute a *search direction*

$$v_n = -dF(u_n)^{-1}F(u_n),$$

and then we find a *step length* $\alpha_n$ such that

$$u_{n + 1} = u_n + \alpha_n\cdot v_n$$

is a better guess.

In [ ]:
firedrake.solve(F == 0, u, bc)

Now that we've found a solution, we'd like to get a single number to show how much it differs from the solution of the linear problem.
The `norm` function calculates the $L^2$ norm of its argument:

$$\|u\|_{L^2} = \left(\int_\Omega|u|^2\, dx\right)^{1/2}$$

This is a quick way to get a feel for how much two fields differ.

In [ ]:
firedrake.norm(u - u_l) / firedrake.norm(u_l)

Finally we'll plot the two solutions, the linear one grey and almost transparent, and the nonlinear one in solid colors.
(Matplotlib isn't great for this so you might have to rotate it around a bit.)

In [ ]:
fig = plt.figure()
axes = fig.add_subplot(projection='3d')
firedrake.trisurf(u_l, alpha=0.25, cmap='Greys_r', axes=axes)
firedrake.trisurf(u, axes=axes)